In this project, we use Detectron2 to train the Faster-RCNN model on speed limit sign dataset. 

Detectron2 is Facebook AI Research's next generation library that provides state-of-the-art detection and segmentation algorithms. It is the successor of Detectron and maskrcnn-benchmark.

![img](https://github.com/facebookresearch/detectron2/raw/main/.github/Detectron2-Logo-Horz.svg)

# Install detectron2

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Properly install detectron2. (Please do not install twice in both ways)
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

# Data loading

In [ ]:
from collections import ChainMap
SPEED_THING_CLASSES = ['pl100', 'pl120', 'pl20', 'pl30', 'pl40', 'pl50', 'pl60', 'pl70', 'pl80']
SPEED_THING_DATASET_ID_TO_CONTIGUOUS_ID =  dict(
    ChainMap(*[{i: i} for i in range(9)]))

In [ ]:
import os

# Detectron imports
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import register_coco_instances
    
def setup_speed_dataset(dataset_dir):
    train_image_dir = os.path.join(dataset_dir, 'data')
    test_image_dir = os.path.join(dataset_dir, 'data')

    train_json_annotations = os.path.join(
        dataset_dir, 'train_dataset.json')
    test_json_annotations = os.path.join(
        dataset_dir, 'val_dataset.json')

    register_coco_instances(
        "speed_custom_train",
        {},
        train_json_annotations,  
        train_image_dir)
    MetadataCatalog.get(
        "speed_custom_train").thing_classes = SPEED_THING_CLASSES
    MetadataCatalog.get(
        "speed_custom_train").thing_dataset_id_to_contiguous_id = SPEED_THING_DATASET_ID_TO_CONTIGUOUS_ID

    register_coco_instances(
        "speed_custom_val",
        {},
        test_json_annotations,
        test_image_dir)
    MetadataCatalog.get(
        "speed_custom_val").thing_classes = SPEED_THING_CLASSES
    MetadataCatalog.get(
        "speed_custom_val").thing_dataset_id_to_contiguous_id = SPEED_THING_DATASET_ID_TO_CONTIGUOUS_ID

In [ ]:
setup_speed_dataset("export_dataset")

# Training

In [1]:
import os
import sys


# Detectron imports
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.engine import DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator, DatasetEvaluators, verify_results


class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        """
        Builds evaluators for post-training mAP report.
        Args:
            cfg(CfgNode): a detectron2 CfgNode
            dataset_name(str): registered dataset name

        Returns:
            detectron2 DatasetEvaluators object
        """
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluators = [COCOEvaluator(dataset_name, cfg, True, output_folder)]
        return DatasetEvaluators(evaluators)

    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        """
        Builds DataLoader for test set.
        Args:
            cfg(CfgNode): a detectron2 CfgNode
            dataset_name(str): registered dataset name

        Returns:
            detectron2 DataLoader object specific to the test set.
        """
        return build_detection_test_loader(
            cfg, dataset_name)

    @classmethod
    def build_train_loader(cls, cfg):
        """
        Builds DataLoader for train set.
        Args:
            cfg(CfgNode): a detectron2 CfgNode

        Returns:
            detectron2 DataLoader object specific to the train set.
        """
        return build_detection_train_loader(
            cfg)


In [5]:
#load cfg for detectron2 from a config yaml file
from detectron2.config import get_cfg
cfg = get_cfg()
cfg.merge_from_file("model/configs/resnet.yaml")
cfg.OUTPUT_DIR = "model/output"
trainer = Trainer(cfg)
trainer.train()

Command Line Args: Namespace(config_file='model/configs/resnet.yaml', dataset_dir='export_dataset', dist_url='tcp://127.0.0.1:51118', eval_only=False, image_corruption_level=0, inference_config='', iou_correct=0.5, iou_min=0.1, machine_rank=0, min_allowed_score=0.0, num_gpus=1, num_machines=1, opts=[], random_seed=0, resume=False, savefigdir='./savefig', test_dataset='', visualize=0)
[12/15 22:23:32 detectron2]: Rank of current process: 0. World size: 1
[12/15 22:23:33 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
sys.platform            linux
Python                  3.7.9 (default, Aug 31 2020, 12:42:55) [GCC 7.3.0]
numpy                   1.21.6
detectron2              0.6 @/home/hew/.lo